In [1]:
import numpy as np
import pylab as pl
import arrayfire as af
af.set_backend("cpu")

In [2]:
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20  
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8     
pl.rcParams['xtick.minor.size'] = 4     
pl.rcParams['xtick.major.pad']  = 8     
pl.rcParams['xtick.minor.pad']  = 8     
pl.rcParams['xtick.color']      = 'k'     
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'    

pl.rcParams['ytick.major.size'] = 8     
pl.rcParams['ytick.minor.size'] = 4     
pl.rcParams['ytick.major.pad']  = 8     
pl.rcParams['ytick.minor.pad']  = 8     
pl.rcParams['ytick.color']      = 'k'     
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in' 

In [3]:
# Setting velocity and spatial grid points
N_positions = 500
ghost_zones = 3
N_velocity  = 500

In [4]:
# Boundaries of domain
left_boundary  = 0
right_boundary = 1.0
length         = right_boundary - left_boundary

In [5]:
# Setting mass of the particle, boltzmann-constant
mass_particle      = 1.0
boltzmann_constant = 1.0

In [6]:
# Scattering time scale
tau   = 0.01
# Magnitude of maximum velocity
v_max = 5.0

In [7]:
# Time Parameters for the simulation:
dt         = 0.001 # Size of the time-step
final_time = 10.0
time       = np.arange(dt, final_time, dt)

In [8]:
# Setting up the temperature parameters for the simulations:
T_left   = 0.24055809477
T_right  = 0.22802902812

In [9]:
# Setting up of the density parameters for the simulation:
rho_left  = 1.0
rho_right = 0.92307692

In [10]:
# Bulk velocity parameters for left and right states:
v_bulk_l = 1.0
v_bulk_r = 1.08333333

In [11]:
# Setting up of spatial and velocity grids:
x  = np.linspace(left_boundary, right_boundary, N_positions)
dx = x[1] - x[0]

In [12]:
# Obtaining the coordinates for the ghost-zones:
x_ghost_left  = np.linspace(-(ghost_zones)*dx + left_boundary, left_boundary - dx, ghost_zones)
x_ghost_right = np.linspace(right_boundary + dx, right_boundary + ghost_zones*dx , ghost_zones)

In [13]:
# Combining them to obtain the entire spatial grid
x  = np.concatenate([x_ghost_left, x, x_ghost_right])

In [14]:
# Obtaining the velocity grid
v  = np.linspace(-v_max, v_max, N_velocity)
x  = af.to_array(x)
v  = af.to_array(v)

v_bulk = af.constant(0, N_positions + 2*ghost_zones, N_velocity)
v_bulk[:ghost_zones + 0.5*N_positions, :] = v_bulk_l
v_bulk[ghost_zones + 0.5*N_positions:, :]  = v_bulk_r

In [15]:
# Conversion to allow for easy vectorization
x = af.tile(x, 1, N_velocity)
v = af.tile(af.reorder(v), N_positions + 2*ghost_zones, 1)

In [16]:
def calculate_density(f, v):
    deltav           = af.sum(v[0, 1]-v[0, 0])
    value_of_density = af.sum(f, 1)*deltav
    af.eval(value_of_density)
    return(value_of_density)

In [17]:
def calculate_temperature(f, v):
    deltav               = af.sum(v[0, 1]-v[0, 0])
    value_of_temperature = af.sum(f*(v - v_bulk)**2, 1)*deltav
    value_of_temperature = value_of_temperature/calculate_density(f, v)
    af.eval(value_of_temperature)
    return(value_of_temperature)

In [18]:
def f_MB(x, v, f):
    n = af.tile(calculate_density(f, v), 1, N_velocity)
    T = af.tile(calculate_temperature(f, v), 1, N_velocity)
    f_MB = n*af.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T))*\
             af.exp(-mass_particle*(v - v_bulk)**2/(2*boltzmann_constant*T))
    af.eval(f_MB)
    return(f_MB)


In [19]:
def f_interp(dt, x, v, f):
    x_new     = x - v*dt
    step_size = af.sum(x[1,0] - x[0,0])
    f_inter   = af.constant(0, N_positions + 2*ghost_zones, N_velocity)
    
    f_inter   = af.Array.as_type(f_inter, af.Dtype.f64)
    
    
    f_inter[ghost_zones:-ghost_zones,:] = af.approx1(f, (x_new[ghost_zones:-ghost_zones,:]/step_size), af.INTERP.CUBIC)
      
    f_left    = rho_left * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_left))*\
                af.exp(-mass_particle*(v - v_bulk_l)**2/(2*boltzmann_constant*T_left))
    
    f_right   = rho_right * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_right))*\
                af.exp(-mass_particle*(v - v_bulk_r)**2/(2*boltzmann_constant*T_right))
    
    f_inter[ghost_zones:-ghost_zones,:] = af.select(x_new[ghost_zones:-ghost_zones, :]<=left_boundary, \
                                                     f_left[ghost_zones:-ghost_zones,:], \
                                                     f_inter[ghost_zones:-ghost_zones,:] \
                                                    )
    
    f_inter[ghost_zones:-ghost_zones,:] = af.select(x_new[ghost_zones:-ghost_zones, :]>=right_boundary, \
                                                     f_right[ghost_zones:-ghost_zones,:], \
                                                     f_inter[ghost_zones:-ghost_zones,:] \
                                                    )
    af.eval(f_inter)
    return f_inter


In [20]:
# Intializing the values for f
f_initial = af.constant(0, N_positions + 2*ghost_zones, N_velocity)

In [21]:
f_initial[:ghost_zones + (N_positions/2),:] = \
(rho_left * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_left))*\
af.exp(-mass_particle*(v - v_bulk_l)**2/(2*boltzmann_constant*T_left)))[:ghost_zones + (N_positions/2),:]
    
f_initial[(N_positions/2) + ghost_zones:, :] = \
(rho_right * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_right))*\
af.exp(-mass_particle*(v - v_bulk_r)**2/(2*boltzmann_constant*T_right)))[(N_positions/2) + ghost_zones:, :]

In [22]:
# Initializing the starting value for the simulation
f_current = f_initial

In [23]:
for time_index, t0 in enumerate(time):
    print("Computing For Time Index = ", time_index)
    #print("Physical Time            = ", t0)
    # We shall split the Boltzmann-Equation and solve it:
    # In this step we are solving the collisionless equation
    f_current = af.Array.as_type(f_current, af.Dtype.f64)
    fstar = f_interp(dt, x, v, f_current)
    
    fstar[:ghost_zones,:] = \
    (rho_left * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_left))*\
     af.exp(-mass_particle*(v - v_bulk_l)**2/(2*boltzmann_constant*T_left)))[:ghost_zones,:]
        
    
    fstar[N_positions + ghost_zones:, :] = \
    (rho_right * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_right))*\
     af.exp(-mass_particle*(v - v_bulk_r)**2/(2*boltzmann_constant*T_right)))[N_positions + ghost_zones:, :]
    
    # We turn off the term v(df/dx) for the following two steps
    f0             = f_MB(x, v, fstar)
    f_new          = f0 + (fstar - f0)*np.exp(-dt/tau)
    
    f_new[:ghost_zones,:] = \
    (rho_left * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_left))*\
     af.exp(-mass_particle*(v - v_bulk_l)**2/(2*boltzmann_constant*T_left)))[:ghost_zones,:]
        
    
    f_new[N_positions + ghost_zones:, :] = \
    (rho_right * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_right))*\
     af.exp(-mass_particle*(v - v_bulk_r)**2/(2*boltzmann_constant*T_right)))[N_positions + ghost_zones:, :]
    

    f_current = f_new
    
    pl.xlabel('$x$')
    pl.ylabel(r'$\rho$')
    pl.plot(x[3:-3,0], calculate_density(f_current[ghost_zones:-ghost_zones, :], v[ghost_zones:-ghost_zones, :]))
    pl.title('Time = ' + str(t0))
    pl.savefig('images/' + '%04d'%time_index + '.png')
    pl.clf()

Computing For Time Index =  0
Computing For Time Index =  1
Computing For Time Index =  2
Computing For Time Index =  3
Computing For Time Index =  4
Computing For Time Index =  5
Computing For Time Index =  6
Computing For Time Index =  7
Computing For Time Index =  8
Computing For Time Index =  9
Computing For Time Index =  10
Computing For Time Index =  11
Computing For Time Index =  12
Computing For Time Index =  13
Computing For Time Index =  14
Computing For Time Index =  15
Computing For Time Index =  16
Computing For Time Index =  17
Computing For Time Index =  18
Computing For Time Index =  19
Computing For Time Index =  20
Computing For Time Index =  21
Computing For Time Index =  22
Computing For Time Index =  23
Computing For Time Index =  24
Computing For Time Index =  25
Computing For Time Index =  26
Computing For Time Index =  27
Computing For Time Index =  28
Computing For Time Index =  29
Computing For Time Index =  30
Computing For Time Index =  31
Computing For Time

KeyboardInterrupt: 

In [ ]:
pl.plot(x[3:-3,0], calculate_density(f_current[ghost_zones:-ghost_zones, :], v[ghost_zones:-ghost_zones, :]))
#pl.plot(temp)